In [26]:
# %pip install bertopic
# %pip install indicnlp-library
# %pip install transformers


In [27]:
# different params to try


from bertopic.representation import MaximalMarginalRelevance
representation_model = MaximalMarginalRelevance(diversity=0.2)



In [28]:
# get processed data

import os
import pandas as pd

file_path = "../data/"

data = pd.read_csv(file_path+"train.csv")



In [29]:
# Convert to list
df=data["news_title"]

docs = [str(i) for i in df.values]

docs=docs[:10000]


In [30]:


# Use model to convert words to vectors - EMBEDDING STEP

import transformers
import numpy as np
from tqdm import tqdm
import torch 
from transformers.pipelines import pipeline

model_name = "ai4bharat/indic-bert"

# Use tokenizer to separate words

tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, keep_accents=True)

# Use model to convert words to vectors - EMBEDDING STEP

model = pipeline("feature-extraction", model=model_name, tokenizer=tokenizer)


# model = transformers.AutoModel.from_pretrained(model_name)

# n = len(data)
# batch_size = 8

# embeds = torch.zeros((n, model.config.hidden_size))

# docs = [str(i) for i in data["news_title"].values]

# docs=docs[:10000]

# for i in tqdm(range(0, n, batch_size)):
#     i_end = min(i+batch_size, n)
#     batch = docs[i:i_end]
#     batch_encoded = tokenizer.batch_encode_plus(batch, padding=True, truncation=True, return_tensors='pt')
#     batch_inputs = {'input_ids': batch_encoded['input_ids'], 'attention_mask': batch_encoded['attention_mask']}
#     batch_outputs = model(**batch_inputs)
#     with torch.no_grad():
#         batch_embed = batch_outputs.last_hidden_state[:, 0, :]
#     embeds[i:i_end,:] = batch_embed


Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'sop_classifier.classifier.bias', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=80, min_samples=40,
                        prediction_data=True, gen_min_span_tree=True)

In [32]:
# Tokenize docs trivially (split on spaces)

from indicnlp.tokenize import sentence_tokenize, indic_tokenize

def tokenize_ta(text,return_tensors="pt",*args,**kwargs):
    return indic_tokenize.trivial_tokenize(text)

In [33]:
# Common list of stopwords

stopwords=['அங்கு',
 'அங்கே',
 'அடுத்த',
 'அதனால்',
 'அதன்',
 'அதற்கு',
 'அதிக',
 'அதில்',
 'அது',
 'அதே',
 'அதை',
 'அந்த',
 'அந்தக்',
 'அந்தப்',
 'அன்று',
 'அல்லது',
 'அவன்',
 'அவரது',
 'அவர்',
 'அவர்கள்',
 'அவள்',
 'அவை',
 'ஆகிய',
 'ஆகியோர்',
 'ஆகும்',
 'இங்கு',
 'இங்கே',
 'இடத்தில்',
 'இடம்',
 'இதனால்',
 'இதனை',
 'இதன்',
 'இதற்கு',
 'இதில்',
 'இது',
 'இதை',
 'இந்த',
 'இந்தக்',
 'இந்தத்',
 'இந்தப்',
 'இன்னும்',
 'இப்போது',
 'இரு',
 'இருக்கும்',
 'இருந்த',
 'இருந்தது',
 'இருந்து',
 'இவர்',
 'இவை',
 'உன்',
 'உள்ள',
 'உள்ளது',
 'உள்ளன',
 'எந்த',
 'என',
 'எனக்',
 'எனக்கு',
 'எனப்படும்',
 'எனவும்',
 'எனவே',
 'எனினும்',
 'எனும்',
 'என்',
 'என்ன',
 'என்னும்',
 'என்பது',
 'என்பதை',
 'என்ற',
 'என்று',
 'என்றும்',
 'எல்லாம்',
 'ஏன்',
 'ஒரு',
 'ஒரே',
 'ஓர்',
 'கொண்ட',
 'கொண்டு',
 'கொள்ள',
 'சற்று',
 'சிறு',
 'சில',
 'சேர்ந்த',
 'தனது',
 'தன்',
 'தவிர',
 'தான்',
 'நான்',
 'நாம்',
 'நீ',
 'பற்றி',
 'பற்றிய',
 'பல',
 'பலரும்',
 'பல்வேறு',
 'பின்',
 'பின்னர்',
 'பிற',
 'பிறகு',
 'பெரும்',
 'பேர்',
 'போது',
 'போன்ற',
 'போல',
 'போல்',
 'மட்டுமே',
 'மட்டும்',
 'மற்ற',
 'மற்றும்',
 'மிக',
 'மிகவும்',
 'மீது',
 'முதல்',
 'முறை',
 'மேலும்',
 'மேல்',
 'யார்',
 'வந்த',
 'வந்து',
 'வரும்',
 'வரை',
 'வரையில்',
 'விட',
 'விட்டு',
 'வேண்டும்',
 'வேறு']

from sklearn.feature_extraction.text import CountVectorizer

# Create a vectorizer object to generate term document counts for topic representation - TOKENIZATION STEP

vectorizer_model = CountVectorizer(
    stop_words=stopwords,analyzer='word',
    tokenizer=tokenize_ta
)

In [34]:

# Create a BERTopic model

from bertopic import BERTopic

topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    verbose=True,
    calculate_probabilities=False,
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
)




In [35]:
# Fit the model on the documents
# embeds_np = embeds.detach().numpy()

topics = topic_model.fit_transform(docs)



100%|██████████| 10000/10000 [07:30<00:00, 22.22it/s] 
2023-03-21 13:41:16,423 - BERTopic - Transformed documents to Embeddings
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2023-03-21 13:41:29,950 - BERTopic - Reduced dimensionality
2023-03-21 13:41:30,187 - BERTopic - Clustered reduced embeddings


In [36]:
# Get the topics

topic_model.get_topic_info()




,Topic,Count,Name
0,-1,1060,-1_டைமிங்_டுடே_ஷாட்_பலி
1,0,3490,0_பலி_கொலை_தடை_பெண்
2,1,3292,1_நாளை_கொள்ளை_திடீர்_காங்கிரஸ்
3,2,487,2_ஐ_சி_ஆஸி_கே
4,3,385,3_கொள்ளை_அன்கோ_காலிறுதியில்_தடை
5,4,290,4_பிடிபட்டனர்_சிக்கினர்_தேர்தல்_உயர்வு
6,5,273,5_சிக்கியது_அத்வானி_ஒரிசா_படுகாய
7,6,189,6_யில்_பல்கலை_வாசன்_காங்
8,7,171,7_கருணாநிதி_ஜெயலலிதா_தேமுதிக_இன்று
9,8,136,8_கிளிப்பிங்ஸ்_சானியா_ஜோடி_தோல்வி


In [37]:
# Get the words in a topic

topic_model.get_topic(0)


[('பலி', 0.02485119694926554),
 ('கொலை', 0.020649496889679746),
 ('தடை', 0.014595126540341095),
 ('பெண்', 0.011397240348477628),
 ('மோதல்', 0.01116206066515714),
 ('புதிய', 0.008816756629877527),
 ('திட்ட', 0.008545539602224754),
 ('அருகே', 0.00832744008187064),
 ('அனுமதி', 0.0080158491968024),
 ('முடிவு', 0.007529881274711517)]

In [38]:
topic_model.get_representative_docs(0)


['மனைவியை குத்தி கொன்ற ராணுவ அதிகாரி கைது',
 'வெளிநாட்டு பெண் கடத்தல் ரயில்வே போலீசார் கைது',
 'தீக்குளித்து பெண் பலி கணவர் கைது']

In [39]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents
cleaned_docs = topic_model._preprocess_text(docs)

# Extract vectorizer and tokenizer from BERTopic
vectorizer = topic_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(topic_model.get_topics())-1)]


In [40]:

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

0.4367870096380995

In [41]:
topic_model.visualize_barchart()
